#
### **Парсер: подготовка данных**

In [1]:
from pandas import read_csv 
from data.labels import labels


path = 'data/'
label1 = labels['label1']

In [2]:
train = read_csv(path + 'train.csv', index_col='id')
train.head()

,text,label
id,,
809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта
854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта
4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта


In [3]:
tar = read_csv(path + 'target.csv', index_col='id')
tar.head()

,start,stop,search
id,,,
809436509,1279,1343,Размер обеспечения исполнения контракта 6593.2...
854885310,1222,1318,Поставщик должен предоставить обеспечение испо...
4382157,1297,1343,Размер обеспечения исполнения контракта 10.00%
184555082,1304,1350,Размер обеспечения исполнения контракта 10.00%
211645258,1302,1348,Размер обеспечения исполнения контракта 10.00%


#
#### **nerpa extractor**
`Extractor nerpa` сделан на [yargy](https://github.com/natasha/yargy), подсветка реализована с помощью [ipymarkup](https://github.com/natasha/ipymarkup).
Все инструменты из open source проекта `natasha`  

In [4]:
import nerpa
from rules.rules import(
    Contract, Guarantee,
    Amount,
    Addition
)

exone = nerpa.Extractor(
    [Contract, Amount, Addition])
extwo = nerpa.Extractor(
    [Guarantee, Amount, Addition])

def switch_extractor(label):
    return exone if label == label1 else extwo

In [5]:
for id, _ in train.iloc[:1].iterrows():

    lb = _['label']
    txt = _['text']

    ex_ = switch_extractor(label=lb)
             
    nerpack = nerpa.NERpack()
    nerpack.add_marks(
        entities=ex_(txt), text=txt)
    
    nerpack.show_full(txt)
    print(nerpack.marks)

[(1203, 1235, 'Обеспечение исполнения контракта', '1'), (1246, 1278, 'обеспечение исполнения контракта', '1'), (1279, 1285, 'Размер', 'ADD'), (1286, 1318, 'обеспечения исполнения контракта', '1'), (1319, 1343, '6593.25 Российский рубль', '₽'), (1352, 1384, 'обеспечения исполнения контракта', '1'), (1399, 1431, 'обеспечению Исполнение контракта', '1'), (1777, 1809, 'обеспечения исполнения контракта', '1'), (2104, 2127, 'исполнения обязательств', '1')]


In [6]:
for id, _ in train.iloc[61:64].iterrows():

    lb = _['label']
    txt = _['text']

    ex_ = switch_extractor(label=lb)
             
    nerpack = nerpa.NERpack()
    nerpack.add_marks(entities=ex_(txt))
    
    nerpack.show(txt)
    print(nerpack.entities) 
    print(nerpack.markstr() + '\n')    #   show mark's tags 

['label1', 'amount', 'addition']
ADD₽11ADD1₽11



['label2', 'amount', 'addition']
22ADD2₽22ADD222222



['label1', 'amount', 'addition']
ADD11ADD111ADD1₽11



#
#### **false negative/positive errors**

Находим пустые записи в `extracted_part`

In [7]:
length = tar['stop'] - tar['start']

answ_on = tar[length != 0].index
answ_off = tar[length == 0].index

Определяем ошибки 1 и 2 рода, где парсер нашел пункт, хотя ответа нет и где не нашел, но ответ есть

В идеале оба массива ошибок должны быть пустыми, но в приоритете избавиться от ошибок типа *false negative* (II), так как мы сразу отбросим эти записи и больше не сможем с ними работать

Ошибки типа *false positive* (I) могут отсеяться на следующих этапах парсинга

In [8]:
one_check = nerpa.Extractor([Contract])
two_check = nerpa.Extractor([Guarantee])

In [9]:
label_off = []  # search all labeloff docs

for id, _ in train.iterrows():

    lb =  _['label']
    txt = _['text']

    extractor = (one_check 
                 if lb == label1 
                 else two_check)

    if extractor.empty(txt):
        label_off.append(id) 

In [10]:
label_on = [x for x in train.index 
            if x not in label_off]

In [11]:
errorsI = [x for x in label_on if x in answ_off]
errorsII = [x for x in label_off if x in answ_on]

print(errorsI, errorsII)

[476192036, 588528614, 833988154, 219363951, 471097685, 14015810] []


Корректируем правила для [labels](rules/entities/label.py)

```python
CONTRACT = rule(
    or_(
        normalized('обеспечение'),
        normalized('исполнение'),
    ).repeatable(),
    normalized('настоящий').optional(),
    or_(
        normalized('контракт'),
        normalized('договор'),
    )
)
```

In [ ]:
#      >>>execute 4th, 5th code cells

Создаем новый датасет без записей, где экстрактор не смог извлечь лебл

In [12]:
data = train.loc[label_on]
data.shape

(1498, 2)

#
#### **Search answers**



In [ ]:
amount_check = nerpa.Extractor([Amount])
amount_off = []

for id, _ in data.iterrows():
    txt = _['text']     
    if amount_check.empty(txt):
        amount_off.append(id)

In [ ]:
amount_off == errorsI

True

In [ ]:
data= data.drop(data.loc[amount_off].index)
data.shape

(1492, 2)

In [14]:
def adapt_(entries, len):
    return [(entry, entry+len) for entry in entries]

In [15]:
import re
def find_entries(patterns, str):
    for __ in patterns:
        entries = [_.start() for _ in re.finditer(__, str)]
        if entries:
            return adapt_(
                entries=entries, len=len(__))
    return entries

In [16]:
patterns = ['11222', '1122', '112', '12']    
#   '{1, 2, ADD}{1, 2, ADD}₽₽₽', '{1, 2, ADD}{1, 2, ADD}₽₽'
#   '{1, 2, ADD}{1, 2, ADD}₽', '{1, 2, ADD}₽'

In [36]:
data['entries'] = 0
data['entries'] = data['entries'].astype('object')

In [37]:
for id, _ in data.iterrows():

    lb = _['label']
    txt = _['text']
             
    ex_ = switch_extractor(label=lb)
    nerpack = nerpa.NERpack()
    nerpack.add_marks(entities=ex_(txt))
    
    markstr = (nerpack.markstr()
                    .replace('2', '1')
                    .replace('ADD', '1')
                    .replace('₽', '2') 
                    )
    data.at[id, 'entries'] = find_entries(
        patterns=patterns, str=markstr)

In [45]:
data.head()

,text,label,entries
id,,,
809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,"[(2, 5)]"
854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,"[(0, 3)]"
4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(3, 6)]"
184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(2, 5)]"
211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(2, 5)]"


In [46]:
data['start'] = 0
data['stop'] = 0

In [ ]:
multyentry = []
for id, _ in data.iterrows():

    lb = _['label']
    txt = _['text']
    entries = _['entries']

    if not entries:
        continue
    
    ex_ = switch_extractor(label=lb)
    nerpack = nerpa.NERpack()
    nerpack.add_marks(entities=ex_(txt))
    marks = nerpack.marks
    entries = _['entries']

    if len(entries) == 1:
        entry = entries[0]
        data.at[id, 'start'] = marks[entry[0]].start
        data.at[id, 'stop'] = marks[entry[1]].stop
    else:
        multyentry.append(id)

In [48]:
data.head()

,text,label,entries,start,stop
id,,,,,
809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,"[(2, 5)]",1279,1384
854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,"[(0, 3)]",1252,1365
4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(3, 6)]",1297,1384
184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(2, 5)]",1304,1391
211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,"[(2, 5)]",1302,1389


In [ ]:
# from natasha import (
#     Segmenter,
#     MorphVocab,

#     NewsEmbedding,
#     NewsMorphTagger,
#     NewsSyntaxParser,
#     MoneyExtractor,
#     Doc

# )

# from razdel import tokenize
# from razdel import sentenize

# tokens = list(tokenize(data[0]['text']))
# for _ in tokens:
#     print(i)


# segmenter = Segmenter()
# morph_vocab = MorphVocab()

# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)
# syntax_parser = NewsSyntaxParser(emb)


# text = text[spans[0].start:spans[-1].stop]
# text = 'Способ обеспечения исполнения контракта, гарантийных обязательств'
# doc = Doc(text)

# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)

# sent = doc.sents[0]
# # sent.morph.print()


# sent.syntax.print()